In [1]:
import numpy as np
import pandas as pd

import pyopentd as pt

# 解析実行

In [2]:
# TDモデルの読み込み
dwg_path = "./td_model/sample.dwg"
td = pt.ThermalDesktop(dwg_path)

In [3]:
# ケースセットの一覧確認
casesets = td.get_casesets()
casesets.head()

,group_name,caseset_name
0,chamber,Case Set 0
1,chamber,steady
2,chamber,No rad
3,orbit,Case Set 2


In [4]:
# ケースセットの読み込み
# orbit.Case Set 2 以外は動かないので注意。
group_name = 'orbit'
caseset_name = 'Case Set 2'
case = td.GetCaseSet(caseset_name, group_name)

In [5]:
# 軌道情報の取得
orbit_name = pt.get_orbit_name_from_caseset(case)
print(orbit_name)
df_orbit = td.get_orbit(orbit_name)
df_orbit.head()

sample_orbit


,Times,sun_x,sun_y,sun_z,planet_x,planet_y,planet_z,radius
0,0.000000,0.0,0.0,1.0,0.0,-1.066924e-13,-1.000000e+00,1.078393
1,473.081495,0.0,0.0,1.0,0.0,5.000000e-01,-8.660254e-01,1.078393
2,946.162989,0.0,0.0,1.0,0.0,8.660254e-01,-5.000000e-01,1.078393
3,1419.244484,0.0,0.0,1.0,0.0,1.000000e+00,-1.331770e-13,1.078393
4,1765.927780,0.0,0.0,1.0,0.0,9.272846e-01,3.743571e-01,1.078393


In [6]:
# 実行
case.Run()

# 結果出力

In [7]:
sav_name = case.SindaOptions.SaveFilename
print(sav_name)

case2.sav


In [12]:
sav_path = f'./td_model/{case.UserDirectory}/{sav_name}'
sav_path

'./td_model/output/case2.sav'

In [13]:
savefile = pt.SaveFile(sav_path)

In [14]:
df_temp = savefile.get_all_temperature().drop('Times', axis=1)
df_heat = savefile.get_all_heatrate().drop('Times', axis=1)

In [15]:
def get_attitude(t_list, df):
    attitude_list = []
    for t in t_list:
        attitude = np.zeros(6)
        T_orbit = df.iloc[-1]['Times']
        dt = df.iloc[-1]['Times'] - df.iloc[-2]['Times']
        t = t % T_orbit
        t_bef_index = t // dt
        t_af_index = t_bef_index + 1
        
        attitude_bef = df.iloc[int(t_bef_index)].values
        attitude_af = df.iloc[int(t_af_index)].values
        
        rate = (t % dt) / dt
        attitude = (1-rate)*attitude_bef + rate*attitude_af
        
        attitude_list.append(attitude[1:])
    
    return pd.DataFrame(attitude_list, columns=df.columns[1:])

In [16]:
df_orbit_modify = get_attitude(savefile.times, df_orbit)
# df_orbit_modify
df_time = pd.DataFrame(savefile.times, columns=['Times'])
df_case = pd.DataFrame([f'{case.GroupName}.{case.Name}']*len(savefile.times), columns=['case'])

In [17]:
df = pd.concat([df_case, df_time, df_orbit_modify, df_temp, df_heat], axis=1)
df.to_csv('sample_output.csv', index=False)

In [18]:
df

,case,Times,sun_x,sun_y,sun_z,planet_x,planet_y,planet_z,radius,CHAMBER.T1,...,PANEL_PZ.Q1,PANEL_PZ.Q2,PANEL_PZ.Q3,PANEL_PZ.Q4,PANEL_PZ.Q5,PANEL_PZ.Q6,PANEL_PZ.Q7,PANEL_PZ.Q8,PANEL_PZ.Q9,SPACE.Q1
0,orbit.Case Set 2,0.0,0.0,0.0,1.0,0.0,-1.066924e-13,-1.000000,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,orbit.Case Set 2,120.0,0.0,0.0,1.0,0.0,1.268280e-01,-0.966017,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,orbit.Case Set 2,240.0,0.0,0.0,1.0,0.0,2.536561e-01,-0.932033,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,orbit.Case Set 2,360.0,0.0,0.0,1.0,0.0,3.804841e-01,-0.898050,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,orbit.Case Set 2,480.0,0.0,0.0,1.0,0.0,5.053529e-01,-0.860673,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,orbit.Case Set 2,9600.0,0.0,0.0,1.0,0.0,-6.070576e-01,0.758968,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,orbit.Case Set 2,9720.0,0.0,0.0,1.0,0.0,-6.999022e-01,0.666123,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,orbit.Case Set 2,9840.0,0.0,0.0,1.0,0.0,-7.927468e-01,0.573279,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,orbit.Case Set 2,9960.0,0.0,0.0,1.0,0.0,-8.693000e-01,0.493284,1.078393,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
